In [1]:
import pandas as pd
import numpy as np
import random
import ast
from collections import defaultdict

In [422]:
df = pd.read_csv(R'D:\calorie_RL\data\meal3_mealTime.csv')
df

,meal_name,ingredients,calories_per_serving,carbs,proteins,fats,diet_type,meal_time,serving_size,cost_per_serving_in_inr,region,meal_time_categories
0,Chole Bhature,"Chickpeas, Refined Flour, Spices, Onion",450,55,15.0,18.0,Vegetarian,breakfast/lunch,1 plate,50,North Indian,"['breakfast', 'lunch']"
1,Aloo Paratha,"Wheat Flour, Potato, Spices, Butter",290,35,6.0,12.0,Vegetarian,breakfast/lunch,1 paratha,25,North Indian,"['breakfast', 'lunch']"
2,Butter Chicken,"Chicken, Butter, Cream, Tomato, Spices",490,18,28.0,32.0,Non-Vegetarian,dinner,1 bowl,80,North Indian,['dinner']
3,Kadhi Pakora,"Besan, Curd, Spices, Onion",260,20,8.0,16.0,Vegetarian,lunch,1 bowl,30,North Indian,['lunch']
4,Litti Chokha,"Wheat Flour, Sattu, Brinjal, Potato, Tomato",360,50,10.0,12.0,Vegetarian,snacks,3 littis with chokha,25,North Indian,['snacks']
...,...,...,...,...,...,...,...,...,...,...,...,...
481,Oats Porridge,"Oats, Milk, Sugar/Honey",150,25,5.0,3.0,Vegetarian,Breakfast,1 bowl,30,North Indian,['breakfast']
482,Sweet Potato Chaat,"Sweet Potato, Spices, Lemon Juice",180,30,3.0,5.0,Vegetarian,Snack,1 bowl,30,North Indian,['snacks']
483,"Fruit Salad (Banana, apple, mango,blueberry)","Banana, apple, mango,blueberry",120,30,2.0,1.0,Vegetarian,Snack,1 bowl,60,North Indian,['snacks']
484,Basmati Rice,"Basmati Rice, Water",220,45,5.0,2.0,Vegetarian,Lunch,1 cup cooked,30,North Indian,['lunch']


In [423]:
df['diet_type'].value_counts()

diet_type
Vegetarian        423
Non-Vegetarian     63
Name: count, dtype: int64

In [424]:
def filter_meals_by_diet(diet_types, meal_time):
    """
    Filters meals based on the provided diet types and meal time.

    Args:
        diet_types (list): List of diet types (e.g., ['Vegetarian', 'Non-Vegetarian']).
        meal_time (str): The meal time to filter (e.g., 'breakfast').

    Returns:
        pd.DataFrame: Filtered DataFrame of meals.
    """
    # Ensure 'meal_time_categories' is a list
    def parse_list(value):
        if isinstance(value, str):
            try:
                return ast.literal_eval(value)  # Safely convert string to list
            except (ValueError, SyntaxError):
                return []  # Return an empty list if parsing fails
        return value

    df['meal_time_categories'] = df['meal_time_categories'].apply(parse_list)
    
    # Filter meals by diet types and meal time
    filtered_meals = df[(df['diet_type'].isin(diet_types)) & 
                        (df['meal_time_categories'].apply(lambda x: meal_time in x))]
    return filtered_meals

In [425]:
# ...existing code...
class DietEnvironment:
    def __init__(self, meals, total_calories, total_budget, meal_times):
        self.meals = meals
        self.total_calories = total_calories
        self.total_budget = total_budget
        self.meal_times = meal_times  # List of meal times (e.g., ['breakfast', 'lunch', 'dinner'])
        self.current_calories = {meal_time: 0 for meal_time in meal_times}
        self.current_budget = 0
        self.selected_meals = {meal_time: [] for meal_time in meal_times}
        self.current_meal_time = 0  # Index to track the current meal time

    def reset(self):
        self.current_calories = {meal_time: 0 for meal_time in self.meal_times}
        self.current_budget = 0
        self.selected_meals = {meal_time: [] for meal_time in self.meal_times}
        self.current_meal_time = 0
        return self.get_state()

    def get_state(self):
        if self.current_meal_time >= len(self.meal_times):
            return (0, self.current_budget)  # Return a default state when out of bounds
        meal_time = self.meal_times[self.current_meal_time]
        return (self.current_calories[meal_time], self.current_budget)

    def step(self, action):
        if self.current_meal_time >= len(self.meal_times):
            return self.get_state(), 0, True  # Return a default state if out of bounds

        meal_time = self.meal_times[self.current_meal_time]
        meal = self.meals.iloc[action]
        calories = meal['calories_per_serving']
        cost = meal['cost_per_serving_in_inr']

        # Add meal to the plan
        self.selected_meals[meal_time].append(meal['meal_name'])
        self.current_calories[meal_time] += calories
        self.current_budget += cost

        # Calculate reward
        if self.current_budget > self.total_budget:
            reward = -20  # Penalty for exceeding budget
            done = True
        elif self.current_calories[meal_time] >= 0.8 * self.total_calories:  # Flexible calorie goal
            reward = 50  # Reward for meeting calorie goal
            self.current_meal_time += 1  # Move to the next meal time
            done = self.current_meal_time >= len(self.meal_times)  # Done if all meal times are covered
        else:
            reward = -10  # Small penalty for not meeting the goal yet
            done = False

        return self.get_state(), reward, done
# ...existing code...

In [426]:
class DietAgent:
    def __init__(self, n_actions):
        self.n_actions = n_actions
        self.q_table = defaultdict(lambda: np.zeros(self.n_actions))
        self.epsilon = 1.0  # Start with high exploration
        self.epsilon_min = 0.1  # Minimum exploration rate
        self.epsilon_decay = 0.995  # Decay rate
        self.alpha = 0.1  # Learning rate
        self.gamma = 0.9  # Discount factor

    def choose_action(self, state):
        state = str(state)
        if random.uniform(0, 1) < self.epsilon:
            return random.randint(0, self.n_actions - 1)  # Explore
        else:
            return np.argmax(self.q_table[state])  # Exploit

    def learn(self, state, action, reward, next_state):
        state = str(state)
        next_state = str(next_state)
        predict = self.q_table[state][action]
        target = reward + self.gamma * np.max(self.q_table[next_state])
        self.q_table[state][action] += self.alpha * (target - predict)

        # Decay epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [427]:
def train_agent(diet_type, total_calories, total_budget, meal_times, episodes=5000):
    meals_by_time = {meal_time: filter_meals_by_diet(diet_type, meal_time) for meal_time in meal_times}
    if any(meals.empty for meals in meals_by_time.values()):
        raise ValueError(f"No meals found for one or more meal times: {meal_times}")

    env = DietEnvironment(pd.concat(meals_by_time.values()), total_calories, total_budget, meal_times)
    agent = DietAgent(len(env.meals))

    for _ in range(episodes):
        state = env.reset()
        done = False

        while not done:
            action = agent.choose_action(state)
            next_state, reward, done = env.step(action)
            agent.learn(state, action, reward, next_state)
            state = next_state

    return agent

In [428]:
def generate_diet_plan(agent, diet_types, total_calories, total_budget, meal_times):
    # Filter meals for each meal_time
    meals_by_time = {meal_time: filter_meals_by_diet(diet_types, meal_time) for meal_time in meal_times}
    
    # Check if any meal_time has no available meals
    for meal_time, meals in meals_by_time.items():
        if meals.empty:
            raise ValueError(f"No meals available for {meal_time}. Please check the dataset or constraints.")
    
    # Initialize a dictionary to store the selected meals
    selected_meals_details = {meal_time: [] for meal_time in meal_times}
    total_selected_calories = 0
    used_meals = set()  # To track meals that have already been selected

    # Step 1: Ensure at least one meal for each meal_time
    for meal_time in meal_times:
        available_meals = meals_by_time[meal_time]
        if available_meals.empty:
            continue

        # Exclude already used meals
        available_meals = available_meals[~available_meals['meal_name'].isin(used_meals)]

        # Shuffle the available meals to introduce randomness
        available_meals = available_meals.sample(frac=1).reset_index(drop=True)

        # Select the highest-calorie meal for the current meal_time
        if not available_meals.empty:
            meal = available_meals.iloc[0]
            selected_meals_details[meal_time].append({
                'meal_name': meal['meal_name'],
                'calories_per_serving': meal['calories_per_serving'],
                'cost_per_serving_in_inr': meal['cost_per_serving_in_inr'],
                'meal_time': meal_time
            })
            used_meals.add(meal['meal_name'])
            total_selected_calories += meal['calories_per_serving']

    # Step 2: Add a second meal to some meal_times if calories are not satisfied
    if total_selected_calories < total_calories:
        remaining_calories = total_calories - total_selected_calories
        for meal_time in meal_times:
            available_meals = meals_by_time[meal_time]
            available_meals = available_meals[~available_meals['meal_name'].isin(used_meals)]
            available_meals = available_meals.sample(frac=1).reset_index(drop=True)  # Shuffle meals

            for _, meal in available_meals.iterrows():
                if len(selected_meals_details[meal_time]) < 2 and remaining_calories > 0:
                    selected_meals_details[meal_time].append({
                        'meal_name': meal['meal_name'],
                        'calories_per_serving': meal['calories_per_serving'],
                        'cost_per_serving_in_inr': meal['cost_per_serving_in_inr'],
                        'meal_time': meal_time
                    })
                    used_meals.add(meal['meal_name'])
                    total_selected_calories += meal['calories_per_serving']
                    remaining_calories -= meal['calories_per_serving']

                # Stop if total calories are satisfied
                if total_selected_calories >= total_calories:
                    break
            if total_selected_calories >= total_calories:
                break

    # Flatten the selected meals into a single DataFrame
    diet_plan_df = pd.DataFrame([meal for meals in selected_meals_details.values() for meal in meals])
    return diet_plan_df

In [433]:
diet_type = ['Vegetarian', 'Non-Vegetarian']
total_calories = 3000
total_budget = 800
meal_times = ['dinner', 'lunch', 'breakfast', 'snacks', 'dessert']

# Train the agent and generate the diet plan
agent = train_agent(diet_type, total_calories, total_budget, meal_times)
diet_plan_details = generate_diet_plan(agent, diet_type, total_calories, total_budget, meal_times)

# Combine all meal plans into a single DataFrame
diet_plan_df = pd.DataFrame()

for meal_time, details in diet_plan_details.items():
    # Ensure details is a dictionary and convert it to a DataFrame
    details_df = pd.DataFrame([details])  # Convert the dictionary to a DataFrame
    details_df['meal_time'] = meal_time  # Add a column for meal time
    diet_plan_df = pd.concat([diet_plan_df, details_df], ignore_index=True)

# Display the final DataFrame

In [434]:
diet_plan_df.T

,0,1,2,3
0,Paneer butter masala,300,50,dinner
1,Mushroom matar (Matar Khumb),180,30,dinner
2,Sauteed Vegetables,150,30,lunch
3,Galho,300,80,lunch
4,Greek Yogurt,150,40,breakfast
5,Pongal,280,35,breakfast
6,Hando Guri,180,25,snacks
7,Jeera Aloo,150,25,snacks
8,Chhena kheeri,200,55,dessert
9,Aval kesari,250,40,dessert
